##Projekt z uczenia maszynowego
#Adam Gontarek 106619

Zależności

In [163]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.externals import joblib
from sklearn.cross_validation import cross_val_predict
from scipy.stats import randint as sp_randint
from time import time
from operator import itemgetter
import numpy as np
from pandas import DataFrame

Zmienne

In [164]:
workspace_path = "C:/development/repos/git/github/ZED/projekt2/"


Zaladowanie zbiorów: treningowy, testowy

In [165]:
train_data_df = pd.read_csv(workspace_path + "data/training_data.txt", sep=";", na_values=["nan"], keep_default_na=False, low_memory=False)
X_test = pd.read_csv(workspace_path + "data/test_data.txt", sep=",", na_values=["nan"], keep_default_na=False)


Filtracja danych [dostosowanie do danych z pogrupowanymi etykietami] 

In [166]:
df_filtered = train_data_df.query('res_name not in ("DA","DC","DT","DU","DG","DI","UNK","UNX","UNL","PR","PD","Y1","EU","N","15P","UQ","PX4","NAN")')
df_grouped = df_filtered.drop_duplicates(['pdb_code', 'res_name'])
df_without_classes = df_grouped.groupby("res_name").filter(lambda x: len(x) >= 5)
print 'Filtered training data dim:', df_without_classes.shape

Filtered training data dim: (11005, 795)


Zaladowanie danych z pogrupowanymi etykietami

In [167]:
group_label_df = pd.read_csv(workspace_path + "data/grouped_res_name.txt", sep=",", na_values=["nan"], keep_default_na=False, low_memory=False)
print 'Group label dim:', group_label_df.shape

Group label dim: (11005, 2)


Wybranie istotnych atrybutow do predykcji klas

In [168]:
part_00_cols = df_without_classes.filter(regex='part_00_.*');
rest_cols = df_without_classes.filter(items=[
  'local_volume', 'local_electrons', 'local_mean', 'local_std',
  'local_max', 'local_skewness', 'local_parts',
  'solvent_mask_count', 'void_mask_count', 'modeled_mask_count', 'solvent_ratio'])

Przygotowanie zbioru testowego

In [169]:
X_test = X_test.fillna(0)
X_test_part_00_cols = X_test.filter(regex='part_00_.*');
X_test_rest_cols = X_test.filter(items=[
  'local_volume', 'local_electrons', 'local_mean', 'local_std',
  'local_max', 'local_skewness', 'local_parts',
  'solvent_mask_count', 'void_mask_count', 'modeled_mask_count', 'solvent_ratio'])
X_test =  pd.concat([X_test_part_00_cols,  X_test_rest_cols], axis=1);

Przygotowanie zbioru pod klasyfikator 1

In [170]:
filtered_df = pd.concat([rest_cols,  part_00_cols], axis=1);
filtered_df = filtered_df.fillna(0);
X_train = filtered_df
y_train = df_without_classes['res_name']

Rozmiar zbioru dla klasyfikatora 1

In [171]:
print "X_train", X_train.shape
print "y_train", y_train.shape

X_train (11005, 80)
y_train (11005L,)


Klasyfikator 1

In [172]:
rfc = RandomForestClassifier(n_jobs=8)

param_grid = {
              "n_estimators": [100],
              # "max_features": [3],
              # "min_samples_split": [5],
              # "min_samples_leaf": [2],
              "bootstrap": [False],
              "criterion": ["gini"]
}

clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='recall_macro')

Obliczenia klasyfikator 1

In [173]:
clf.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(clf.best_estimator_)
print()
print "Best score:", clf.best_score_


Best parameters set found on development set:
()
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=8,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
()
Best score: 0.173626498947


Predykcja klasyfikatora 1 dla dostarczonych przez prowadzącego danych testowych

In [174]:
y_pred =clf.best_estimator_.predict_proba(X_test)

y_pred_df = DataFrame(data=y_pred, columns=clf.best_estimator_.classes_)
y_pred_df.to_csv(workspace_path + "predictions/classifier1_predicts_probability.txt", sep=",")


y_pred =clf.best_estimator_.predict(X_test)

y_pred_df = DataFrame(data=y_pred)
y_pred_df.to_csv(workspace_path + "predictions/classifier1_predicts.txt", sep=",")


y_pred = cross_val_predict(clf.best_estimator_, X_train, y_train, cv=5)
print "Recall score[based on cv]: ", recall_score(y_train, y_pred, average='macro')

Recall score[based on cv]:  0.174720539636


Przygotowanie zbioru pod klasyfikator 2

In [175]:
X_train = filtered_df
y_train = group_label_df['res_name_group']



Rozmiar zbioru dla klasyfikatora 2

In [176]:
print "X_train", X_train.shape
print "y_train", y_train.shape

X_train (11005, 80)
y_train (11005L,)


Klasyfikator 2

In [177]:
rfc = RandomForestClassifier(n_jobs=8)

param_grid = {
              "n_estimators": [100],
              # "max_features": [3],
              # "min_samples_split": [5],
              # "min_samples_leaf": [2],
              "bootstrap": [False],
              "criterion": ["gini"]
}

clf2 = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5, scoring='recall_macro')

Obliczenia klasyfikator 2

In [178]:
clf2.fit(X_train, y_train)
print("Best parameters set found on development set:")
print()
print(clf2.best_estimator_)
print()
print "Best score:", clf2.best_score_

Best parameters set found on development set:
()
RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=8,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
()
Best score: 0.211201429703


Predykcja klasyfikatora 2 dla dostarczonych przez prowadzącego danych testowych

In [179]:
y_pred =clf2.best_estimator_.predict_proba(X_test)

y_pred_df = DataFrame(data=y_pred, columns=clf2.best_estimator_.classes_)
y_pred_df.to_csv(workspace_path + "predictions/classifier2_predicts_probability.txt", sep=",")


y_pred =clf2.best_estimator_.predict(X_test)

y_pred_df = DataFrame(data=y_pred)
y_pred_df.to_csv(workspace_path + "predictions/classifier2_predicts.txt", sep=",")


y_pred = cross_val_predict(clf2.best_estimator_, X_train, y_train, cv=5)
print "Recall score[based on cv]: ", recall_score(y_train, y_pred, average='macro')

Recall score[based on cv]:  0.206720470832


Serializacja klasyfikatorów

In [180]:
joblib.dump(clf.best_estimator_, workspace_path + 'classifiers/classifier1.pkl')
joblib.dump(clf2.best_estimator_, workspace_path + 'classifiers/classifier2.pkl')


['C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_01.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_02.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_03.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_04.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_05.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_06.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_07.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_08.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_09.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_10.npy',
 'C:/development/repos/git/github/ZED/projekt2/classifiers/classifier2.pkl_11.npy',